In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [6]:
%run ../load_anno_mi.py

In [7]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-03 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-03 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-03 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-03 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-03 00:00:34,-1


In [8]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

## Baseline: Fine-tune BERT for classification

In [9]:
num_epochs = 5
seeds = [1, 12, 123]
validation_metric = "f1"

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "pretrained_model_name": "bert-base-uncased",
    "df": anno_mi,
    "feature_name": "utterance_text",
    "label_column": "client_talk_type",
    "seeds": seeds,
    "device": device,
    "batch_size": 8,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "validation_metric": validation_metric,
    "verbose": False,
}

## Focal Loss

In [11]:
loss = "focal"
gamma = 2

In [12]:
bert_classifier = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_focal.csv",
    **kwargs,
)

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.671098,0.639938,0.414010
2,No log,0.623863,0.663069,0.539196
3,No log,0.719978,0.668466,0.520446
4,No log,0.794235,0.661527,0.546534
5,No log,0.868729,0.663840,0.555466


  0%|          | 0/889 [00:00<?, ?it/s]

...

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.646488,0.607172,0.480245
2,No log,0.639824,0.650367,0.518248
3,No log,0.684399,0.639772,0.543672
4,No log,0.870881,0.667482,0.534701
5,No log,0.898170,0.663407,0.530834


  0%|          | 0/1100 [00:00<?, ?it/s]

saving the results dataframe to CSV in client_talk_type_output/bert_classifier_focal.csv


In [13]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.637165,0.490209,"[0.7681200453001132, 0.3871829105473965, 0.315...",0.493358,"[0.752357182473655, 0.42028985507246375, 0.307...",0.489035,"[0.7845575477154425, 0.3589108910891089, 0.323...",1,focal,2,True
0,0.616667,0.497380,"[0.7516858983965232, 0.4008583690987124, 0.339...",0.490963,"[0.7800933125972006, 0.41771019677996424, 0.27...",0.518075,"[0.7252747252747253, 0.38531353135313534, 0.44...",12,focal,2,True
0,0.627395,0.512491,"[0.7561890472618155, 0.4339869281045752, 0.347...",0.507320,"[0.7857811038353602, 0.4598337950138504, 0.276...",0.535636,"[0.728744939271255, 0.41089108910891087, 0.467...",123,focal,2,True


In [14]:
bert_classifier["f1"].mean()

0.5000266858119725

In [15]:
bert_classifier["precision"].mean()

0.49721340933941444

In [16]:
bert_classifier["recall"].mean()

0.5142486864842258

In [17]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.758665  , 0.40734274, 0.33407232])

In [18]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.77274387, 0.43261128, 0.28628508])

In [19]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.7461924 , 0.3850385 , 0.41151515])

## Using Cross-Entropy loss

In [20]:
loss = "cross_entropy"
gamma = None

In [21]:
bert_classifier_ce = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_ce.csv",
    **kwargs,
)

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.819342,0.643793,0.319440
2,No log,0.776637,0.660756,0.427186
3,No log,0.782890,0.676947,0.489899
4,No log,0.850129,0.666924,0.510591
5,No log,0.882697,0.661527,0.509059


  0%|          | 0/889 [00:00<?, ?it/s]

...

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.780773,0.671557,0.406399
2,No log,0.781099,0.655257,0.432979
3,No log,0.786806,0.690302,0.496486
4,No log,0.812554,0.692747,0.536896
5,No log,0.850325,0.673187,0.536569


  0%|          | 0/1100 [00:00<?, ?it/s]

saving the results dataframe to CSV in client_talk_type_output/bert_classifier_ce.csv


In [22]:
bert_classifier_ce

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,seed,loss,gamma,k_fold
0,0.666284,0.411718,"[0.7948684036503108, 0.40261437908496733, 0.03...",0.494171,"[0.7323909334633195, 0.4265927977839335, 0.323...",0.423396,"[0.8689994216310005, 0.3811881188118812, 0.02]",1,cross_entropy,None,True
0,0.660728,0.483504,"[0.7848518111964874, 0.4240035041611914, 0.241...",0.509268,"[0.7467362924281984, 0.4519140989729225, 0.329...",0.472439,"[0.8270676691729323, 0.39933993399339934, 0.19...",12,cross_entropy,None,True
0,0.646169,0.490161,"[0.7708947885939037, 0.41975308641975306, 0.27...",0.501798,"[0.7495219885277247, 0.4335971855760774, 0.322...",0.482520,"[0.7935222672064778, 0.4067656765676568, 0.247...",123,cross_entropy,None,True


In [23]:
bert_classifier_ce["f1"].mean()

0.46179440822554924

In [24]:
bert_classifier_ce["precision"].mean()

0.5017456883387937

In [25]:
bert_classifier_ce["recall"].mean()

0.4594516561739073

In [26]:
np.stack(bert_classifier_ce["f1_scores"]).mean(axis=0)

array([0.78353833, 0.41545699, 0.1863879 ])

In [27]:
np.stack(bert_classifier_ce["precision_scores"]).mean(axis=0)

array([0.74288307, 0.43736803, 0.32498597])

In [28]:
np.stack(bert_classifier_ce["recall_scores"]).mean(axis=0)

array([0.82986312, 0.39576458, 0.15272727])